In [125]:
import pandas as pd
import numpy as np

import pylab as pl
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

### Draft Year Data

In [126]:
def dfplayerYear(year):
    dfplayerYr=pd.read_csv('Data/draft_year_player/dfplayer'+str(year)+'.csv')
    namePlayer=[]
    
    for elem in dfplayerYr['url']:
        namePlayer.append(elem[11:-5])

    dfplayerYr['name']=namePlayer

    del dfplayerYr['Unnamed: 0']
    del dfplayerYr['url']
    del dfplayerYr['draft']
    del dfplayerYr['shot']
    del dfplayerYr['pos']
    
    return dfplayerYr

In [127]:
fundam = []
for yr in range(1996,2016):
    fundam.append(dfplayerYear(yr))

In [128]:
df_all_fundam = pd.concat(fundam)
df_all_fundam=df_all_fundam.reset_index(drop=True)

df_all_fundam = df_all_fundam[df_all_fundam.height!= 'ml>']
df_all_fundam=df_all_fundam.reset_index(drop=True)
weight_all=df_all_fundam['weight']
height_all=df_all_fundam['height']

df_all_fundam[['weight','height']]=df_all_fundam[['weight','height']].astype(float)

### Height Weight Data

In [129]:
def dfplayerAge(age):
    df_age_yr=pd.read_csv('Data/AgeData/df_age_'+str(age)+'.csv',index_col=0)
    return df_age_yr

In [130]:
df_age_frames=[]
for age in range(19,40):
    df_age_frames.append(dfplayerAge(age))

df_ages = pd.concat(df_age_frames)
df_ages=df_ages.reset_index(drop=True)
del df_ages['age_x']


In [131]:
#calculate goals all 
df_ages['goals_all']=df_ages['goals_ev']+df_ages['goals_pp']+df_ages['goals_sh']

#sort by name and age
df_ages_sorted=df_ages.sort(['name', 'age_y'], ascending=[0, 1])

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [132]:
df_ages_sorted.head()

,TOT,age_y,assists_ev,assists_pp,assists_sh,blocks,corsi_against,corsi_for,corsi_pct,corsi_rel_pct,...,shot_pct,shot_pct.1,shots_attempted,shots_attempted.1,takeaways,time_on_ice_avg,uniqueseason,zs_defense_pct,zs_offense_pct,goals_all
5478,CHI,30.0,2.0,1.0,0.0,35.0,312.0,275.0,46.8,-7.0,...,5.9,5.9,68.0,NaN,6.0,14.58,1,54.2,45.8,2.0
416,MIN,20.0,2.0,0.0,0.0,2.0,73.0,40.0,35.4,-8.8,...,0.0,0.0,16.0,16.0,2.0,11.02,1,48.3,51.7,0.0
894,MIN,21.0,1.0,0.0,0.0,5.0,216.0,183.0,45.9,-6.6,...,11.8,11.8,58.0,58.0,4.0,11.16,1,58.4,41.6,4.0
1516,MIN,22.0,0.0,1.0,0.0,11.0,259.0,198.0,43.3,-5.0,...,10.0,10.0,68.0,68.0,9.0,12.59,1,42.7,57.3,4.0
2221,MIN,23.0,5.0,0.0,0.0,13.0,547.0,641.0,54.0,2.6,...,16.9,16.9,193.0,193.0,21.0,15.04,1,46.2,53.8,21.0


In [133]:
# reset index
df_ages_sorted=df_ages_sorted.reset_index(drop=True)

#df_ages_sorted['goals_all']=df_ages_sorted['goals_all']/df_ages_sorted['games_played']
goals_all_data=list(df_ages_sorted['goals_all'])

# you want to predict the next year data
goals_all_data.pop(0)
goals_all_data.append(0)

df_ages_sorted['goals_all']=goals_all_data
df_ages=df_ages_sorted.groupby("name").apply(lambda x: x.iloc[:-1]).reset_index(drop=True)



### Combined Data (All Data)

In [134]:
df_all = pd.merge(df_ages, df_all_fundam, on='name')
df_all=df_all.drop_duplicates()

In [135]:
df_all['goals_ev']=df_all['goals_ev']/df_all['games_played']
df_all['goals_pp']=df_all['goals_pp']/df_all['games_played']
df_all['goals_sh']=df_all['goals_sh']/df_all['games_played']

df_all['assists_all']=df_all['assists_ev']+df_all['assists_pp']+df_all['assists_sh']
df_all['assists_all']=df_all['assists_all']/df_all['games_played']
df_all['assists_ev']=df_all['assists_ev']/df_all['games_played']
df_all['assists_pp']=df_all['assists_pp']/df_all['games_played']
df_all['assists_sh']=df_all['assists_sh']/df_all['games_played']

df_all['zs_offense_pct']=df_all['zs_offense_pct']/100.0
df_all['shot_pct']=df_all['shot_pct']/100.0
df_all['takeaways']=df_all['takeaways']/df_all['games_played']
df_all['hits']=df_all['hits']/df_all['games_played']

df_all['giveaways']=df_all['giveaways']/df_all['games_played']
df_all['blocks']=df_all['blocks']/df_all['games_played']

df_all['shots_attempted']=df_all['shots_attempted']/df_all['games_played']

In [136]:
df_all.head()

,TOT,age_y,assists_ev,assists_pp,assists_sh,blocks,corsi_against,corsi_for,corsi_pct,corsi_rel_pct,...,shots_attempted.1,takeaways,time_on_ice_avg,uniqueseason,zs_defense_pct,zs_offense_pct,goals_all,height,weight,assists_all
0,DET,20.0,0.000000,0.0,0.000000,0.000000,18.0,19.0,51.4,-10.0,...,8.0,0.500000,12.13,1,40.0,0.600,0.000000,188.0,218.0,0.000000
1,DET,21.0,0.000000,0.0,0.000000,0.000000,9.0,22.0,71.0,14.6,...,7.0,0.000000,9.18,1,33.3,0.667,0.060000,188.0,218.0,0.000000
2,DET,22.0,0.060000,0.0,0.000000,0.400000,376.0,409.0,52.1,-2.2,...,141.0,0.140000,10.35,1,33.7,0.663,0.094595,188.0,218.0,0.060000
3,DET,23.0,0.148649,0.0,0.013514,0.527027,628.0,765.0,54.9,-0.4,...,226.0,0.229730,12.18,1,41.2,0.588,0.098765,188.0,218.0,0.162162
4,DET,24.0,0.172840,0.0,0.000000,0.518519,698.0,794.0,53.2,-3.2,...,196.0,0.197531,12.19,1,40.7,0.593,0.208333,188.0,218.0,0.172840


### Normalize Data

In [147]:
df_all_nn=df_all[['goals_all','goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_y','blocks',
                 'giveaways','hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']]

In [148]:
cols_to_norm = ['goals_ev','goals_pp','goals_sh','assists_ev','assists_pp','assists_sh','shots_attempted','age_y','blocks','giveaways',
               'hits','shot_pct','shots_attempted','takeaways','zs_offense_pct','height','weight']
df_all_nn=df_all_nn[cols_to_norm].apply(lambda x: (x - x.mean()) / (x.max() - x.min()))

In [151]:
df_all_nn.head()

,goals_ev,goals_pp,goals_sh,assists_ev,assists_pp,assists_sh,shots_attempted,shots_attempted,age_y,blocks,giveaways,hits,shot_pct,shots_attempted,shots_attempted,takeaways,zs_offense_pct,height,weight
0,-0.108012,-0.032897,-0.008061,-0.177088,-0.062086,-0.020775,0.092108,0.092108,-0.290891,-0.203475,-0.136367,0.173870,-0.080549,0.092108,0.092108,0.079107,0.091405,0.047091,0.125788
1,-0.108012,-0.032897,-0.008061,-0.177088,-0.062086,-0.020775,0.046275,0.046275,-0.238260,-0.203475,0.030300,0.031013,-0.080549,0.046275,0.046275,-0.213862,0.158405,0.047091,0.125788
2,-0.048012,-0.032897,-0.008061,-0.117088,-0.062086,-0.020775,-0.016057,-0.016057,-0.185628,-0.091814,-0.089700,0.251013,-0.042549,-0.016057,-0.016057,-0.131830,0.154405,0.047091,0.125788
3,-0.013418,-0.032897,-0.008061,-0.028439,-0.062086,0.046792,0.005397,0.005397,-0.132997,-0.056354,-0.046277,0.179662,-0.026549,0.005397,0.005397,-0.079254,0.079405,0.047091,0.125788
4,-0.009247,-0.032897,-0.008061,-0.004248,-0.062086,-0.020775,-0.052746,-0.052746,-0.080365,-0.058730,-0.078754,0.077751,-0.014549,-0.052746,-0.052746,-0.098121,0.084405,0.047091,0.125788


In [145]:
df_all_nn['age_y'] = df_all_nn['age_y'].apply(lambda x: abs(x))

In [150]:
df_all_nn=df_all_nn.reset_index(drop=True)

### Ordinary Linear Regression

In [152]:

features = ['goals_pp','goals_ev','goals_sh','assists_ev','assists_pp','assists_sh','age_y',
               'hits','shots_attempted','takeaways','zs_offense_pct','weight','height']


results_all_OLR=[]

result=[]
for k in range(0,100):
    test_idx = np.random.uniform(0, 1, len(df_all_nn_2)) <= 0.8
    train = df_all_nn_2[test_idx==True]
    test = df_all_nn_2[test_idx==False]

    train_goals_all=list(train['goals_all'])
    train_data=train[features].as_matrix()
    
    test_data=test[features].as_matrix()
    

    model=sm.OLS(train_goals_all,train_data).fit()
    
    

    preds = model.predict(test_data)
    test_goals_all=list(test['goals_all'])
    distance=0
    for i in range(len(preds)):
        if preds[i]<0:
            distance += pow((test_goals_all[i]), 2)
        else:    
            distance += pow((test_goals_all[i]-preds[i]), 2)


    result.append(distance)
    
    
    
print sum(result)/(len(result)*len(preds))
    

0.0111416275429


### Ridge Regression

In [153]:
alpha_val=[1e-8, 1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 1e-2]
results_all_ridge=[]

for alphaval in alpha_val:
    result=[]
    
    for k in range(0,20):
        test_idx = np.random.uniform(0, 1, len(df_all_nn_2)) <= 0.8
        train = df_all_nn_2[test_idx==True]
        test = df_all_nn_2[test_idx==False]

        train_goals_all=list(train['goals_all'])
        train_data=train[features].as_matrix()
        #train_data=np.array(train_data).tolist()
        test_data=test[features].as_matrix()
        #test_data=np.array(test_data).tolist()

        clfridge = Lasso(alpha=alphaval).fit(train_data,train_goals_all)

        preds = clfridge.predict(test_data)
        test_goals_all=list(test['goals_all'])
        distance=0
        for i in range(len(preds)):
            distance += pow((test_goals_all[i]-preds[i]), 2)

        
        result.append(distance)
    
    results_all_ridge.append([alphaval, sum(result)/(len(result)*len(preds))])
    
print results_all_ridge
    

[[1e-08, 0.0067748772088938617], [1e-06, 0.0066924040033363129], [1e-05, 0.0070231922831623054], [0.0001, 0.0072430058698100958], [0.0005, 0.0071075383619926599], [0.001, 0.0074974683019051426], [0.01, 0.016450736144722807]]


In [102]:
results_df_ridge = pd.DataFrame(results_all_ridge, columns=["Penalty", "error"])

pl.plot(results_df_ridge.Penalty, results_df_ridge.error)
pl.title("Error with Increasing Penalty (Ridge)")
pl.xlabel('Penalty')
pl.ylabel('Error')


In [103]:
results_df_ridge

,Penalty,error
0,1.000000e-08,0.007068
1,1.000000e-06,0.006900
2,1.000000e-05,0.006930
3,1.000000e-04,0.007009
4,5.000000e-04,0.007764
5,1.000000e-03,0.007702
6,1.000000e-02,0.016585


### Lasso Regression

In [158]:
alpha_val=[1e-8, 1e-6, 1e-5, 1e-4, 5e-4, 1e-3, 1e-2]

results_all_lasso=[]
dict_coef={}
dict_coef_sum={}
for i in range(len(features)+3):
    dict_coef[i]=0
    dict_coef_sum[i]=0
    
for alphaval in alpha_val:
    result=[]
    
    for k in range(0,100):
        test_idx = np.random.uniform(0, 1, len(df_all_nn_2)) <= 0.8
        train = df_all_nn_2[test_idx==True]
        test = df_all_nn_2[test_idx==False]

        train_goals_all=list(train['goals_all'])
        train_data=train[features].as_matrix()
        
        test_data=test[features].as_matrix()
        
        clflasso = Lasso(alpha=alphaval).fit(train_data,train_goals_all)
        coeffs=clflasso.coef_
        
        for elem in range(len(coeffs)):
            if coeffs[elem]==0:
                dict_coef[elem]=dict_coef[elem]+1
            dict_coef_sum[elem]=dict_coef_sum[elem]+coeffs[elem]/float(800)
            
        preds = clflasso.predict(test_data)
        test_goals_all=list(test['goals_all'])
        distance=0
        for i in range(len(preds)):
            distance += pow((test_goals_all[i]-preds[i]), 2)

        
        result.append(distance)
    
    results_all_lasso.append([alphaval, sum(result)/(len(result)*len(preds))])
    
print results_all_lasso
    

[[1e-08, 0.0070264260758363768], [1e-06, 0.0069109866827471046], [1e-05, 0.0067446240323634412], [0.0001, 0.0070143203781889828], [0.0005, 0.0072845643061446872], [0.001, 0.0076703926422647924], [0.01, 0.015545898582913517]]


In [159]:
dict_coef_sum

{0: 0.43587929553646698,
 1: 0.35012697919274932,
 2: 0.079767631967595592,
 3: 0.035178946290364742,
 4: -0.0067041064069161289,
 5: -0.0075079589239249091,
 6: -0.0051773466067234035,
 7: -0.011968763428794712,
 8: 0.13289965689625532,
 9: 9.6633093615484463e-17,
 10: 2.899022632796463e-18,
 11: 1.7683393086238644e-18,
 12: 0.08676137788563501,
 13: 0.021650384148029302,
 14: 0.0019356043498908353,
 15: -0.011009136794598499}

In [160]:
dict(zip(train[features].columns, dict_coef_sum.values()))

{'age_y': -0.0051773466067234035,
 'assists_ev': 0.035178946290364742,
 'assists_pp': -0.0067041064069161289,
 'assists_sh': -0.0075079589239249091,
 'goals_ev': 0.35012697919274932,
 'goals_pp': 0.43587929553646698,
 'goals_sh': 0.079767631967595592,
 'height': -0.011009136794598499,
 'hits': -0.011968763428794712,
 'shots_attempted': 1.7683393086238644e-18,
 'takeaways': 0.08676137788563501,
 'weight': 0.0019356043498908353,
 'zs_offense_pct': 0.021650384148029302}

In [161]:
dict_coef.values()

[100, 100, 326, 100, 413, 399, 312, 224, 5, 313, 457, 466, 100, 289, 385, 273]

In [162]:
results_df_lasso = pd.DataFrame(results_all_lasso, columns=["Penalty", "error"])

pl.plot(results_df_lasso.Penalty, results_df_lasso.error)
pl.title("Error with Increasing Penalty (Lasso)")
pl.xlabel('Penalty')
pl.ylabel('Error')
